# Tool Calling in LangChain

* Tool Creation
* Tool Binding
    * Test Tool Binding Result
* Append Tool Result for LLM Execution

In [1]:
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

## **Step 1:** Tools Creation

In [ ]:
# tool create
import os
from typing import Annotated
from langchain_core.tools import InjectedToolArg
EXCHANGE_RATE_API_KEY = os.getenv("EXCHANGE_RATE_API_KEY")

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """ This tool fetches the conversion factor between two currencies using an external API.
    """
    url = f"https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/pair/{base_currency}/{target_currency}"
    response = requests.get(url)
    return response.json()["conversion_rate"]

@tool
def convert_currency(base_currency_value: int, conversion_factor: Annotated[float, InjectedToolArg]) -> float:
    """ This function converts the base currency to the target currency using the conversion factor.
    """
    return round(base_currency_value * conversion_factor,2)

### Tools Testing to see if they work individually

In [4]:
get_conversion_factor.invoke({'base_currency':'USD', 'target_currency':'PKR'})

282.0785

In [7]:
convert_currency.invoke({'base_currency_value':10, 'conversion_factor':282.0785})

2820.79

## **Step 2:** Tool Binding

In [4]:
# Create LLM Instance
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Bind the tool to the LLM
llm_with_tools = llm.bind_tools([get_conversion_factor, convert_currency])

In [5]:
# Use "Messages" to use the LLM with tools

query = HumanMessage('What is the conversion factor between USD and PKR, and based on that can you convert 10 USD to PKR')
messages = [query]
print(messages)

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 10 USD to PKR', additional_kwargs={}, response_metadata={})]


In [6]:
result = llm_with_tools.invoke(messages)

# LLM will call "Tool" instead of giving us some text 
print(result)
messages.append(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_uONDeZsHtRaU0Sta9BdYm1Lj', 'function': {'arguments': '{"base_currency":"USD","target_currency":"PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 119, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVKRhXbHX7FHk2ILpQbjUqBlV3N2d', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-a27b49b0-845e-4be9-9677-1ce54abdd1f0-0' tool_calls=[{'name': 'get_conversion_factor', 'args': {'base_currency': 'USD', 'target_currency': 'PKR'}, 'id': 'call_uONDeZsHtRaU0Sta9BdYm1Lj', 'type': 'tool_call'}] usage_metadata={'input_tokens': 119, 'output_tokens': 23, 'total_

In [7]:
result.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'PKR'},
  'id': 'call_uONDeZsHtRaU0Sta9BdYm1Lj',
  'type': 'tool_call'}]

In [8]:
# Messages contains the tool call with its specific format
messages

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 10 USD to PKR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uONDeZsHtRaU0Sta9BdYm1Lj', 'function': {'arguments': '{"base_currency":"USD","target_currency":"PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 119, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVKRhXbHX7FHk2ILpQbjUqBlV3N2d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a27b49b0-845e-4be9-9677-1ce54abdd1f0-0', tool_calls=[{'name': 'get_conversion_factor', 'a

## **Step 2.1:** Tool Call 

In [9]:
for tool_call in result.tool_calls:
    selected_tool = {"get_conversion_factor": get_conversion_factor, "convert_currency": convert_currency}[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What is the conversion factor between USD and PKR, and based on that can you convert 10 USD to PKR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uONDeZsHtRaU0Sta9BdYm1Lj', 'function': {'arguments': '{"base_currency":"USD","target_currency":"PKR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 119, 'total_tokens': 142, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVKRhXbHX7FHk2ILpQbjUqBlV3N2d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a27b49b0-845e-4be9-9677-1ce54abdd1f0-0', tool_calls=[{'name': 'get_conversion_factor', 'a

In [10]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2XAvfQsC4zhhyLLYHBHuknYR', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":282.0785}', 'name': 'convert_currency'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 154, 'total_tokens': 179, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0392822090', 'id': 'chatcmpl-BVKSNQrUr5iN2iCXAjID14tQezgIm', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9c9f0820-440f-409b-a378-44b9c7d94c40-0', tool_calls=[{'name': 'convert_currency', 'args': {'base_currency_value': 10, 'conversion_factor': 282.0785}, 'id': 'call_2XAvfQsC4zhhyLLYHBHuknYR', 'type': 'tool_call'}], usage_metadata={'input_tokens': 154, 'output